# Project 2, Part 2, Parse Peak's sales JSON file into CSV files

University of California, Berkeley

Master of Information and Data Science (MIDS) program

w205 - Fundamentals of Data Engineering


# Included Modules and Packages

Code cell containing your includes for modules and packages

In [14]:
import csv

import json

import math
import numpy as np
import pandas as pd

import psycopg2

# Supporting code

Code cells containing any supporting code, such as connecting to the database, any functions, etc.  

Remember you can freely use any code from the labs. You do not need to cite code from the labs.

In [15]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

In [16]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [17]:
cursor = connection.cursor()

In [18]:
def my_read_csv_file(file_name, limit):
    "read the csv file and print only the first limit rows"
    
    csv_file = open(file_name, "r")
    
    csv_data = csv.reader(csv_file)
    
    i = 0
    
    for row in csv_data:
        i += 1
        if i <= limit:
            print(row)
            
    print("\nPrinted ", min(limit, i), "lines of ", i, "total lines.")

In [19]:
def my_recursive_print_json(j, level = -1):
    "given a json object print it"
    
    level += 1
    
    spaces = "    "
    
    if type(j) is dict:
        dict_2_list = list(j.keys())
        for k in dict_2_list:
            print(spaces * level + k)
            my_recursive_print_json(j[k], level)
            
    elif type(j) is list:
        for (i, l) in enumerate(j):
            print(spaces * level + "[" + str(i) + "]")
            my_recursive_print_json(l, level)
                  
    else:
        print(spaces * level + "value:", str(j))
                  


In [20]:
def my_read_nested_json(file_name):
    "given a file of json, read it and parse it meaningfully"
    
    f = open(file_name, "r")
    
    j = json.load(f)
    
    f.close
    
    my_recursive_print_json(j)

# 2.2.1 Recursive walk of Peak's sales JSON file to help understand the structure

Peak has send AGM a nested JSON file of sales data for October 3, 2020.  We need to first understand the structure of this file so we can parse it into CSV files to load into staging tables.

Use the function my_read_nested_json() which calls my_recursive_print_json() to take a recursive walk of the nested JSON file peak_sales_2020_10_03.json to help understand the structure.  These functions are from the lab, and are included above for your convenience.

### Please take some time to study the structure until you understand it.  Understanding the structure will make it much easier to write the parsing code.

Pattern your code after the examples in the labs.

In [27]:
my_read_nested_json("peak_sales_2020_10_03.json")

TypeError: my_read_nested_json() takes 1 positional argument but 2 were given

# 2.2.2 Parse Peak's nested JSON sales file into CSV files

Write Python code to parse Peak's nested JSON sales file, peak_sales_2020_10_03.json, into CSV files.

The first line of each file should be a list of fields as specified below.

Do NOT sort them.  Keep the csv data rows in the same order as the data appears in the JSON file.

Do NOT remove duplicates.

peak_sales.csv
* sale_id - Peak's sale_id (NOT AGM's sale_id)
* sale_date
* sub_total - should equal the sum of line item quantity x 12
* tax - should be 0 as our items are tax exempt
* total_amount = sub_total + tax

peak_stores.csv
* sale_id - we need this to be able to link this to peak_sales
* location_id - Peak's location_id (NOT AGM's store_id)
* name  
* street 
* city 
* state 
* zip 

peak_customers.csv
* sale_id - we need this to be able to link this to peak_sales
* customer_id - Peak's customer_id (NOT AGM's customer_id)
* first_name
* last_name
* street
* city
* state
* zip

peak_line_items.csv
* sale_id - we need this to be able to link this to peak_sales
* line_item_id - you will need to sequentially number them within each sale starting with 1
* product_id - Peak's product ID (NOT AGM's product_id)
* price - should be 12
* quantity
* taxable - should be 'N' for not taxable

After creating these CSV files, be sure and check them into your GitHub repo.

Pattern your code after the examples in the labs. You may use as many code cells as you wish.

In [22]:
f = open("peak_sales_2020_10_03.json", "r")
j = json.load(f)
f.close() 

sales_json_list = []
peak_stores_json_list = []
peak_customers_json_list = []
peak_line_items_json_list = []

for sales in j['sales']:
    sales_json = {}
    sales_json['sale_id'] = sales['sale_id']
    sales_json['sale_date'] = sales['sale_date']
    sales_json['sub_total'] = sales['sub_total']
    sales_json['tax'] = sales['tax']
    sales_json['total_amount'] = sales_json['sub_total'] + sales_json['tax']

    sales_json_list.append(sales_json)

    stores_json = {}
    stores_json['sale_id'] = sales['sale_id']
    stores_json['location_id'] = sales['pickup_from']['location_id']
    stores_json['name'] = sales['pickup_from']['name']
    stores_json['street'] = sales['pickup_from']['street']
    stores_json['city'] = sales['pickup_from']['city']
    stores_json['state'] = sales['pickup_from']['state']
    stores_json['zip'] = sales['pickup_from']['zip']

    peak_stores_json_list.append(stores_json)
    
    customeres_json = {}
    customeres_json['sale_id'] = sales['sale_id']
    customeres_json['customer_id'] = sales['deliver_to']['customer_id']
    customeres_json['first_name'] = sales['deliver_to']['first_name']
    customeres_json['last_name'] = sales['deliver_to']['last_name']
    customeres_json['street'] = sales['deliver_to']['street']
    customeres_json['city'] = sales['deliver_to']['city']
    customeres_json['state'] = sales['deliver_to']['state']
    customeres_json['zip'] = sales['deliver_to']['zip']
    
    peak_customers_json_list.append(customeres_json)
    i = 1
    for items in sales['line_items']:
        line_items_json = {}
        line_items_json['sale_id'] = sales['sale_id']
        line_items_json['line_item_id'] = i
        line_items_json['product_id'] = items['product_id']
        line_items_json['price'] = items['price']
        line_items_json['quantity'] = items['quantity']
        line_items_json['taxable'] = items['taxable']
        i = i+1
        
        peak_line_items_json_list.append(line_items_json)

f = open("peak_sales.csv", "w")
dw = csv.DictWriter(f, sales_json_list[0].keys())
dw.writeheader()
dw.writerows(sales_json_list)
f.close()

f = open("peak_stores.csv", "w")
dw = csv.DictWriter(f, peak_stores_json_list[0].keys())
dw.writeheader()
dw.writerows(peak_stores_json_list)
f.close()

f = open("peak_customers.csv", "w")
dw = csv.DictWriter(f, peak_customers_json_list[0].keys())
dw.writeheader()
dw.writerows(peak_customers_json_list)
f.close()

f = open("peak_line_items.csv", "w")
dw = csv.DictWriter(f, peak_line_items_json_list[0].keys())
dw.writeheader()
dw.writerows(peak_line_items_json_list)
f.close()


# 2.2.3 Display the CSV file peak_sales.csv

Use the function my_read_csv_file() from the labs (with a limit of 10) to display the CSV file you just created:
* peak_sales.csv

For your convenience the function is provided above.

Pattern your code after the examples in the labs.

The output should look similar to this:
```
['sale_id', 'sale_date', 'sub_total', 'tax', 'total_amount']
['5763728874', '2020-10-03', '12', '0', '12']
['5763729036', '2020-10-03', '72', '0', '72']
['5763728904', '2020-10-03', '24', '0', '24']
['5763728973', '2020-10-03', '96', '0', '96']
['5763728757', '2020-10-03', '108', '0', '108']
['5763729051', '2020-10-03', '144', '0', '144']
['5763729153', '2020-10-03', '24', '0', '24']
['5763728608', '2020-10-03', '96', '0', '96']
['5763728696', '2020-10-03', '84', '0', '84']

Printed  10 lines of  98 total lines.
```

In [23]:
my_read_csv_file('peak_sales.csv',10)

['sale_id', 'sale_date', 'sub_total', 'tax', 'total_amount']
['5763728874', '2020-10-03', '12', '0', '12']
['5763729036', '2020-10-03', '72', '0', '72']
['5763728904', '2020-10-03', '24', '0', '24']
['5763728973', '2020-10-03', '96', '0', '96']
['5763728757', '2020-10-03', '108', '0', '108']
['5763729051', '2020-10-03', '144', '0', '144']
['5763729153', '2020-10-03', '24', '0', '24']
['5763728608', '2020-10-03', '96', '0', '96']
['5763728696', '2020-10-03', '84', '0', '84']

Printed  10 lines of  98 total lines.


# 2.2.4 Display the CSV file peak_stores.csv

Use the function my_read_csv_file() from the labs (with a limit of 10) to display the CSV file you just created:
* peak_stores.csv

For your convenience, the function is provided above.

Pattern your code after the examples in the labs.  

The output should look similar to this:

```
['sale_id', 'location_id', 'name', 'street', 'city', 'state', 'zip']
['5763728874', '12573', 'Acme Gourmet Meals', '3000 Telegraph Ave', 'Berkeley', 'CA', '94705']
['5763729036', '12573', 'Acme Gourmet Meals', '3000 Telegraph Ave', 'Berkeley', 'CA', '94705']
['5763728904', '12573', 'Acme Gourmet Meals', '3000 Telegraph Ave', 'Berkeley', 'CA', '94705']
['5763728973', '12573', 'Acme Gourmet Meals', '3000 Telegraph Ave', 'Berkeley', 'CA', '94705']
['5763728757', '12573', 'Acme Gourmet Meals', '3000 Telegraph Ave', 'Berkeley', 'CA', '94705']
['5763729051', '12573', 'Acme Gourmet Meals', '3000 Telegraph Ave', 'Berkeley', 'CA', '94705']
['5763729153', '12573', 'Acme Gourmet Meals', '3000 Telegraph Ave', 'Berkeley', 'CA', '94705']
['5763728608', '12573', 'Acme Gourmet Meals', '3000 Telegraph Ave', 'Berkeley', 'CA', '94705']
['5763728696', '12573', 'Acme Gourmet Meals', '3000 Telegraph Ave', 'Berkeley', 'CA', '94705']

Printed  10 lines of  98 total lines.
```


In [24]:
my_read_csv_file('peak_stores.csv',10)

['sale_id', 'location_id', 'name', 'street', 'city', 'state', 'zip']
['5763728874', '12573', 'Acme Gourmet Meals', '3000 Telegraph Ave', 'Berkeley', 'CA', '94705']
['5763729036', '12573', 'Acme Gourmet Meals', '3000 Telegraph Ave', 'Berkeley', 'CA', '94705']
['5763728904', '12573', 'Acme Gourmet Meals', '3000 Telegraph Ave', 'Berkeley', 'CA', '94705']
['5763728973', '12573', 'Acme Gourmet Meals', '3000 Telegraph Ave', 'Berkeley', 'CA', '94705']
['5763728757', '12573', 'Acme Gourmet Meals', '3000 Telegraph Ave', 'Berkeley', 'CA', '94705']
['5763729051', '12573', 'Acme Gourmet Meals', '3000 Telegraph Ave', 'Berkeley', 'CA', '94705']
['5763729153', '12573', 'Acme Gourmet Meals', '3000 Telegraph Ave', 'Berkeley', 'CA', '94705']
['5763728608', '12573', 'Acme Gourmet Meals', '3000 Telegraph Ave', 'Berkeley', 'CA', '94705']
['5763728696', '12573', 'Acme Gourmet Meals', '3000 Telegraph Ave', 'Berkeley', 'CA', '94705']

Printed  10 lines of  98 total lines.


# 2.2.5 Display the CSV file peak_customers.csv

Use the function my_read_csv_file() from the labs (with a limit of 10) to display the CSV file you just created:
* peak_customers.csv

For your convenience, the function is provided above.

Pattern your code after the examples in the labs.

The output should look similar to this:

```
['sale_id', 'customer_id', 'first_name', 'last_name', 'street', 'city', 'state', 'zip']
['5763728874', '3728404', 'Darrelle', 'Dohrmann', '46 Farwell Terrace', 'Oakland', 'CA', '94609']
['5763729036', '3729309', 'Moria', 'Greenwood', '8803 Delaware Crossing', 'Berkeley', 'CA', '94705']
['5763728904', '3728508', 'Josiah', 'Hulett', '6755 Melby Plaza', 'Oakland', 'CA', '94612']
['5763728973', '3728534', 'Gayle', 'MacGarrity', '286 Onsgard Center', 'Berkeley', 'CA', '94703']
['5763728757', '3729188', 'Courtenay', 'Shirrell', '75 West Park', 'Emeryville', 'CA', '94608']
['5763729051', '3729276', 'Christian', 'Anyene', '869 Transport Crossing', 'Berkeley', 'CA', '94707']
['5763729153', '3729242', 'Linnell', 'Barr', '521 Fallview Alley', 'Oakland', 'CA', '94602']
['5763728608', '3728705', 'Benedick', 'Staneland', '3852 Laurel Park', 'Berkeley', 'CA', '94704']
['5763728696', '3729340', 'Lanni', 'Pickavant', '481 Moose Pass', 'Oakland', 'CA', '94609']

Printed  10 lines of  98 total lines.
```

In [25]:
my_read_csv_file('peak_customers.csv',10)

['sale_id', 'customer_id', 'first_name', 'last_name', 'street', 'city', 'state', 'zip']
['5763728874', '3728404', 'Darrelle', 'Dohrmann', '46 Farwell Terrace', 'Oakland', 'CA', '94609']
['5763729036', '3729309', 'Moria', 'Greenwood', '8803 Delaware Crossing', 'Berkeley', 'CA', '94705']
['5763728904', '3728508', 'Josiah', 'Hulett', '6755 Melby Plaza', 'Oakland', 'CA', '94612']
['5763728973', '3728534', 'Gayle', 'MacGarrity', '286 Onsgard Center', 'Berkeley', 'CA', '94703']
['5763728757', '3729188', 'Courtenay', 'Shirrell', '75 West Park', 'Emeryville', 'CA', '94608']
['5763729051', '3729276', 'Christian', 'Anyene', '869 Transport Crossing', 'Berkeley', 'CA', '94707']
['5763729153', '3729242', 'Linnell', 'Barr', '521 Fallview Alley', 'Oakland', 'CA', '94602']
['5763728608', '3728705', 'Benedick', 'Staneland', '3852 Laurel Park', 'Berkeley', 'CA', '94704']
['5763728696', '3729340', 'Lanni', 'Pickavant', '481 Moose Pass', 'Oakland', 'CA', '94609']

Printed  10 lines of  98 total lines.


# 2.2.6 Display the CSV file peak_line_items.csv

Use the function my_read_csv_file() from the labs (with a limit of 10) to display the CSV file you just created:
* peak_line_items.csv

For your convenience, the function is provided above.

Pattern your code after the examples in the labs.  

The output should look similar to this:

```
['sale_id', 'line_item_id', 'product_id', 'price', 'quantity', 'taxable']
['5763728874', '1', '42314780', '12', '1', 'N']
['5763729036', '1', '42314677', '12', '1', 'N']
['5763729036', '2', '42314782', '12', '3', 'N']
['5763729036', '3', '42314784', '12', '2', 'N']
['5763728904', '1', '42314780', '12', '1', 'N']
['5763728904', '2', '42314784', '12', '1', 'N']
['5763728973', '1', '42314677', '12', '2', 'N']
['5763728973', '2', '42314780', '12', '2', 'N']
['5763728973', '3', '42314782', '12', '2', 'N']

Printed  10 lines of  353 total lines.
```

In [26]:
my_read_csv_file('peak_line_items.csv',10)

['sale_id', 'line_item_id', 'product_id', 'price', 'quantity', 'taxable']
['5763728874', '1', '42314780', '12', '1', 'N']
['5763729036', '1', '42314677', '12', '1', 'N']
['5763729036', '2', '42314782', '12', '3', 'N']
['5763729036', '3', '42314784', '12', '2', 'N']
['5763728904', '1', '42314780', '12', '1', 'N']
['5763728904', '2', '42314784', '12', '1', 'N']
['5763728973', '1', '42314677', '12', '2', 'N']
['5763728973', '2', '42314780', '12', '2', 'N']
['5763728973', '3', '42314782', '12', '2', 'N']

Printed  10 lines of  353 total lines.
